# Clean-up and preparation of saguaro mortality datasets for subsequent analyses

1. Import historical saguaro inventory data obtained from Desert Botanical Garden:
   "Full history of inventory and measurement data for all saguaros alive and dead.csv"

2. Drop columns not required for analysis.

3. Parse year from Inventory Date column for use in subsequent clean-up.

4. Select inventories where saguaros were marked as dead ("forever dormant").

5. Find saguaros that had more than one inventory in a given year, and remove the extra inventories.

6. Remove inventories reported prior to timespans of interest.

7. Exclude saguaro deaths unrelated to weather analysis.

8. Group inventories into the two timespans of interest.

9. Count the number of saguaros that died in each timespan.

10. Export datasets to csv: Inventories_ready_for_analysis.csv and Mortality_rates_ready_for_analysis.csv

### Import dependencies, read in dataset and store in pandas DataFrame.

In [98]:
# Import dependencies
import pandas as pd
import numpy as py

In [99]:
# Read in saguaro dataset with cp1252 character encoding and store in pandas DataFrame called full_df.
full_data = "Full history of inventory and measurement data for all saguaros alive and dead.csv"
full_df = pd.read_csv(full_data, encoding='cp1252')

full_df

,Accession Planting,Inventory Date,Status,Condition,Inventory Note,Measurement Date,Height,Height Unit,DBH,DBH Unit,Measurement Note
0,2020-0589-10-0,9/17/2020 0:00,Dormant,Good,accessioned,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-0589-10-0,9/18/2020 17:27,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-0588-10-0,9/17/2020 0:00,Dormant,Good,Accessioned,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-0588-10-0,9/18/2020 17:27,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-0587-10-0,9/17/2020 0:00,Dormant,Good,Accessioned,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13850,1947-1548-01-1,6/17/2009 0:00,Alive,NaN,Inventoried,NaN,NaN,NaN,NaN,NaN,NaN
13851,1947-1548-01-1,7/9/2012 0:00,Alive,NaN,NOTE: This inventory date was found to be inva...,NaN,NaN,NaN,NaN,NaN,NaN
13852,1947-1548-01-1,11/28/2012 12:16,Alive,NaN,NOTE: This inventory date was found to be inva...,NaN,NaN,NaN,NaN,NaN,NaN
13853,1947-1548-01-1,12/17/2012 0:00,Dead,NaN,NOTE: This inventory date was found to be inva...,NaN,NaN,NaN,NaN,NaN,NaN


### Drop columns not required for analysis.

In [100]:
# From full_df, drop columns not needed for mortality analysis, leaving only the inventory event columns.
# Save the remaining (inventory) columns as new DataFrame called inventory_df.
inventory_df = full_df.drop(["Condition", "Measurement Date", "Height", "Height Unit", "DBH", "DBH Unit", "Measurement Note", "Inventory Note"], axis = 1)

inventory_df

,Accession Planting,Inventory Date,Status
0,2020-0589-10-0,9/17/2020 0:00,Dormant
1,2020-0589-10-0,9/18/2020 17:27,Dormant
2,2020-0588-10-0,9/17/2020 0:00,Dormant
3,2020-0588-10-0,9/18/2020 17:27,Dormant
4,2020-0587-10-0,9/17/2020 0:00,Dormant
...,...,...,...
13850,1947-1548-01-1,6/17/2009 0:00,Alive
13851,1947-1548-01-1,7/9/2012 0:00,Alive
13852,1947-1548-01-1,11/28/2012 12:16,Alive
13853,1947-1548-01-1,12/17/2012 0:00,Dead


### Parse year from Inventory Date column for use in subsequent clean-up.

In [101]:
# Extract year from Inventory Date column and store in new column called Inventory Year. Drop Inventory Date column as
# it's no longer needed.
inventory_year = inventory_df["Inventory Date"].str.extract(r'(20\d\d)')
inventory_df["Inventory Year"] = inventory_year
inventory_drop_df = inventory_df.drop(["Inventory Date"], axis = 1)

inventory_drop_df

,Accession Planting,Status,Inventory Year
0,2020-0589-10-0,Dormant,2020
1,2020-0589-10-0,Dormant,2020
2,2020-0588-10-0,Dormant,2020
3,2020-0588-10-0,Dormant,2020
4,2020-0587-10-0,Dormant,2020
...,...,...,...
13850,1947-1548-01-1,Alive,2009
13851,1947-1548-01-1,Alive,2012
13852,1947-1548-01-1,Alive,2012
13853,1947-1548-01-1,Dead,2012


### Select inventories where saguaros were marked as dead ("forever dormant").

In [102]:
# From inventory_df, select inventories where a saguaro was marked as dead ("forever dormant)".
# Save the selection as new DataFrame called forever_dormant_df.
forever_dormant_df = inventory_drop_df.loc[full_df["Status"] == "Dead"]
forever_dormant_df.to_csv("Forever Dormant Inventories.csv")

forever_dormant_df

,Accession Planting,Status,Inventory Year
73,2019-0188-01-12,Dead,2019
80,2019-0142-01-2,Dead,2020
403,2018-0026-01-5,Dead,2019
740,2016-0042-01-4,Dead,2018
741,2016-0042-01-4,Dead,2018
...,...,...,...
13830,1966-8295-01-2,Dead,2016
13847,1949-2064-01-1,Dead,NaN
13848,1947-1600-01-1,Dead,NaN
13853,1947-1548-01-1,Dead,2012


### Find saguaros that had more than one inventory in a given year, and remove the extra inventories.

In [103]:
# View DataFrame row count before removing duplicate inventories
rows_before = len(forever_dormant_df)

print(f"Before removing duplicates, there are {rows_before} rows present in the DataFrame.")

Before removing duplicates, there are 323 rows present in the DataFrame.


In [104]:
# Sort DataFrame by year so earliest inventory will be kept when duplicates are dropped
sorted_df = forever_dormant_df.sort_values(by = "Inventory Year")

# Remove duplicate accessions (but keep first duplicate)
unique_df = sorted_df.drop_duplicates(subset="Accession Planting", keep="first", inplace = False)

# View DataFrame row count after removing duplicates
rows_after = len(unique_df)
duplicates_removed = rows_before - rows_after

print(f"After removing {duplicates_removed} duplicates, there are {len(unique_df)} rows present in the DataFrame.")

After removing 48 duplicates, there are 275 rows present in the DataFrame.


### Remove inventories reported prior to timespans of interest.

In [105]:
# Remove inventories prior to 2013; keep 2013 and on.
# used in next step.
twentythirteen_and_on_df = unique_df.loc[pd.to_numeric(unique_df["Inventory Year"]) >= 2013]

twentythirteen_and_on_df

,Accession Planting,Status,Inventory Year
7119,2012-0254-01-10,Dead,2013
7124,2012-0254-01-1,Dead,2013
13275,1978-0500-01-1,Dead,2013
11204,1987-0156-01-5,Dead,2013
13400,1977-0387-01-2,Dead,2013
...,...,...,...
12145,1985-0254-01-24,Dead,2020
8845,2012-0065-01-14,Dead,2020
10885,1990-0214-01-6,Dead,2020
3422,2012-0315-01-36,Dead,2020


### Exclude saguaro deaths unrelated to weather analysis

#### Many saguaros were inventoried as dead for reasons unrelated to weather:
* Saguaros that died in the propagation facilities
* Saguaros that were inventoried as dead many years after the event of their death
* Saguaro IDs that were deactivated because they were never assigned to an actual plant

#### A list of inventories unrelated to the weather analysis was obtained from a project member familiar with the inventory protocol. The list is imported, joined to the DataFrame, and used to select inventories appropriate for the weather analysis. The list is exported to csv for subsequent analysis.

In [106]:
# Import exclusion list and merge it with twentythirteen_and_on_df using a left join to retain all values in 
# twentythirteen_and_on_df.
exclude_df = pd.read_csv("Saguaros to exclude.csv", encoding = "cp1252")
merged_df = pd.merge(twentythirteen_and_on_df, exclude_df, on = "Accession Planting", how = "left")

merged_df

,Accession Planting,Status,Inventory Year,Unnamed: 1,Exclude
0,2012-0254-01-10,Dead,2013,NaN,exclude
1,2012-0254-01-1,Dead,2013,NaN,exclude
2,1978-0500-01-1,Dead,2013,NaN,NaN
3,1987-0156-01-5,Dead,2013,NaN,NaN
4,1977-0387-01-2,Dead,2013,NaN,NaN
...,...,...,...,...,...
164,1985-0254-01-24,Dead,2020,NaN,NaN
165,2012-0065-01-14,Dead,2020,NaN,NaN
166,1990-0214-01-6,Dead,2020,NaN,NaN
167,2012-0315-01-36,Dead,2020,NaN,NaN


In [107]:
# Select the rows that do not require exclusion and store result to a new DataFrame called include_df.
include_df = merged_df.loc[merged_df["Exclude"].isnull()]

# Drop recently merged columns as they are no longer needed.
inventories_ready_for_analysis_df = include_df.drop(["Unnamed: 1", "Exclude"], axis = 1)
                             
# Export inventories to csv for subsequent analysis.
inventories_ready_for_analysis_df.to_csv("Inventories_ready_for_analysis.csv")

inventories_ready_for_analysis_df

,Accession Planting,Status,Inventory Year
2,1978-0500-01-1,Dead,2013
3,1987-0156-01-5,Dead,2013
4,1977-0387-01-2,Dead,2013
5,2012-0254-01-31,Dead,2013
7,2013-0266-01-15,Dead,2013
...,...,...,...
164,1985-0254-01-24,Dead,2020
165,2012-0065-01-14,Dead,2020
166,1990-0214-01-6,Dead,2020
167,2012-0315-01-36,Dead,2020


### Group inventories into the two timespans of interest

In [108]:
# From include_df, select the inventories from each timespan of interest and store in new DataFrames called
# timespan_1_df and timespan_2_df.

# Timespan 1
timespan_1 = ["2013", "2014"]
timespan_1_df = forever_dormant_df[forever_dormant_df["Inventory Year"].isin(timespan_1)]

# Timespan 2
timespan_2 = ["2015", "2016", "2017", "2018", "2019"]
timespan_2_df = forever_dormant_df[forever_dormant_df["Inventory Year"].isin(timespan_2)]

timespan_1_df

,Accession Planting,Status,Inventory Year
1171,2014-0302-01-1,Dead,2014
1858,2013-0266-01-15,Dead,2013
1859,2013-0266-01-15,Dead,2013
3411,2012-0315-01-37,Dead,2014
4185,2012-0314-01-2,Dead,2013
6481,2012-0280-01-19,Dead,2013
6614,2012-0254-01-9,Dead,2014
6730,2012-0254-01-44,Dead,2013
6731,2012-0254-01-44,Dead,2013
6732,2012-0254-01-44,Dead,2013


### Count the number of saguaros that died in each timespan and export to csv for subsequent analysis

In [109]:
# Count up the number of saguaros lost in each timespan and divide by the number of years in each timespan to find
# the mortality rate for each timespan
mortality_rate_timespan_1 = len(timespan_1_df)/len(timespan_1)
mortality_rate_timespan_2 = len(timespan_2_df)/len(timespan_2)

print(f"{mortality_rate_timespan_1} saguaros were lost per year in the years 2013 and 2014.")
print(f"{mortality_rate_timespan_2} saguaros were lost per year in the years 2015, 2016, 2017, 2018, and 2019.")

15.5 saguaros were lost per year in the years 2013 and 2014.
30.6 saguaros were lost per year in the years 2015, 2016, 2017, 2018, and 2019.


In [110]:
# Append timespans and mortality rates to a list and export list to csv for subsequent analysis
mortality_rates_list = []
mortality_rates_list.append({"Timespan": "1", "Mortality Rate": mortality_rate_timespan_1})
mortality_rates_list.append({"Timespan": "2", "Mortality Rate": mortality_rate_timespan_2})

# Convert list to DataFrame
mortality_rates_df = pd.DataFrame(mortality_rates_list)

# Export list to csv
mortality_rates_df.to_csv("Mortality_rates_ready_for_analysis.csv")

### End of mortality rate data prep